In [1]:
from __future__ import division, print_function
import os
import pandas as pd
import warnings
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from keras import layers
from keras import backend as K
from keras.layers import Input
from keras.layers.core import Activation, Dense, Flatten
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras.layers import Conv2D
from keras.regularizers import l2
from keras.layers.core import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.utils.layer_utils import get_source_inputs
from keras.utils.data_utils import get_file
from keras.utils import layer_utils
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator #이미지 데이터를 tensor로 변한하기 위해 활용되는 라이브러리입니다.
import glob
warnings.filterwarnings('ignore')
# tf.random.set_seed(49)

In [2]:
WEIGHTS_PATH = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5'
def vgg16(weights_path, top=True):

    model = tf.keras.models.Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add( Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add( MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add( Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add( Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add( MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    if top:
        # Classification layer
        model.add(layers.Flatten())
        model.add(layers.Dense(4096, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(4096, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(365, activation='softmax'))
        
    # else:
    #     model.add( GlobalAveragePooling2D())
    
    model.load_weights(filepath=weights_path)

    sgd = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')

    return model

In [3]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       featurewise_center=True,
                                       featurewise_std_normalization=True,
                                       zoom_range=0.2,
                                       channel_shift_range=0.1,
                                       rotation_range=20,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       horizontal_flip=True
                                   )

In [43]:
train_path = 'C:\\Users\\pc\\Desktop\\DLmodel_database\\Trainingdataset'
# test_path = "C:\Users\pc\Desktop\photo O\photo_0_500_in\\korea_url54"
image_size = (224,224)
train_data = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    shuffle=True,
    batch_size=344,
    class_mode = 'categorical',
    color_mode='rgb',
    # seed=49
)

Found 344 images belonging to 4 classes.


In [12]:
help(train_data)

Help on DirectoryIterator in module keras.preprocessing.image object:

class DirectoryIterator(BatchFromFilesMixin, Iterator)
 |  DirectoryIterator(directory, image_data_generator, target_size=(256, 256), color_mode='rgb', classes=None, class_mode='categorical', batch_size=32, shuffle=True, seed=None, data_format=None, save_to_dir=None, save_prefix='', save_format='png', follow_links=False, subset=None, interpolation='nearest', keep_aspect_ratio=False, dtype=None)
 |  
 |  Iterator capable of reading images from a directory on disk.
 |  
 |  Deprecated: `tf.keras.preprocessing.image.DirectoryIterator` is not
 |  recommended for new code. Prefer loading images with
 |  `tf.keras.utils.image_dataset_from_directory` and transforming the output
 |  `tf.data.Dataset` with preprocessing layers. For more information, see the
 |  tutorials for [loading images](
 |  https://www.tensorflow.org/tutorials/load_data/images) and
 |  [augmenting images](
 |  https://www.tensorflow.org/tutorials/image

In [5]:
#없앨 인덱스 번호
# del_folder = glob.glob("C:\\Users\\pc\\Desktop\\photo O\\photo_0_500_in\\korea_url*")

# num = len("C:\\Users\\pc\\Desktop\\photo O\\photo_0_500_in\\korea_url")
# a=[]
# b=[]
# c=[]
# for i in del_folder:
#     if len(i[num:])==1:
#         a.append(i)
#     elif len(i[num:])==2:
#         b.append(i)
#     else:
#         c.append(i)
# d=[]
# for i in a:
#     d.append(i)
# for i in b:
#     d.append(i)
# for i in c:
#     d.append(i)
# del_index = [i[num:] for i in d]
# len(del_index)


In [5]:
# 딥러닝 돌릴 url
folder = glob.glob("C:\\Users\\pc\\Desktop\\0-200\\0-200\\plus_url*")
num = len("D:\\나눔\\지석 800-1000\\plus_url")
a=[]
b=[]
c=[]
for i in folder:
    if len(i[num:])==1:
        a.append(i)
    elif len(i[num:])==2:
        b.append(i)
    else:
        c.append(i)

d=[]
for i in a:
    d.append(i)
for i in b:
    d.append(i)
for i in c:
    d.append(i)

In [7]:
len(d)

0

In [7]:
# test_path = folder[0]
# test_data = train_datagen.flow_from_directory(
#     test_path,
#     target_size=image_size,
#     shuffle=True,
#     batch_size=32,
#     class_mode = 'categorical'
# )

In [8]:
tf.keras.backend.set_image_data_format('channels_last')
weights_file = 'vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5'
weights_path = tf.keras.utils.get_file(weights_file, WEIGHTS_PATH_NO_TOP)
base_model = vgg16(weights_path, top=False)
base_model.trainable = False
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

In [9]:
# tf.keras.backend.set_image_data_format('channels_last')
# weights_file = 'vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5'
# weights_path = tf.keras.utils.get_file(weights_file, WEIGHTS_PATH_NO_TOP)
# base_model = vgg16(weights_path, top=False)
# base_model.trainable = False
# model = tf.keras.models.Sequential()
# model.add(base_model)
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(1024, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(2048, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(4096, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(4, activation='softmax'))

In [10]:
# tf.keras.backend.set_image_data_format('channels_last')
# weights_file = 'vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5'
# weights_path = tf.keras.utils.get_file(weights_file, WEIGHTS_PATH_NO_TOP)
# base_model = vgg16(weights_path, top=False)
# base_model.trainable = False
# model = tf.keras.models.Sequential()
# model.add(base_model)
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(1024, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(2048, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(4096, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(2048, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(1024, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(4, activation='softmax'))

In [11]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
checkpoint = ModelCheckpoint("checkpoints/vgg16_12.h5", monitor='accuracy', verbose=1, save_best_only=True )
early = EarlyStopping(monitor='accuracy', min_delta=0, patience=10, verbose=1, mode='auto')

history = model.fit(
        train_data,
        callbacks=[checkpoint, early],
        epochs=100,
        batch_size=32,
        verbose=1)
#러닝레이트 0.01에서 0.0001로 늘리고 base model의 globalpooling을 주석

Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 2.4174 - accuracy: 0.2820
Epoch 1: accuracy improved from -inf to 0.28198, saving model to checkpoints\vgg16_12.h5
11/11 [==============================] - 20s 1s/step - loss: 2.4174 - accuracy: 0.2820
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 1.6903 - accuracy: 0.3721
Epoch 2: accuracy improved from 0.28198 to 0.37209, saving model to checkpoints\vgg16_12.h5
11/11 [==============================] - 7s 616ms/step - loss: 1.6903 - accuracy: 0.3721
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 1.4665 - accuracy: 0.4157
Epoch 3: accuracy improved from 0.37209 to 0.41570, saving model to checkpoints\vgg16_12.h5
11/11 [==============================] - 7s 628ms/step - loss: 1.4665 - accuracy: 0.4157
Epoch 4/100
11/11 [==============================] - ETA: 0s - loss: 1.3761 - accuracy: 0.4360
Epoch 4: accuracy improved from 0.41570 to 0.43605, saving model to checkpoints\vg

In [6]:
# model.save("room.h5")
model.evaluate(train_data)

11/11 [==============================] - 17s 836ms/step - loss: 0.1425 - accuracy: 0.9506


[0.14250613749027252, 0.9505813717842102]

In [10]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from keras.models import load_model

In [11]:
model = load_model("room.h5")

In [12]:
model.evaluate(train_data)

11/11 [==============================] - 20s 877ms/step - loss: 0.1809 - accuracy: 0.9360


[0.18093939125537872, 0.9360465407371521]

In [45]:
batch = next(train_data)
images,label= batch# 0번 이미지데이터 1번 레이블
labels = model.predict(images)

11/11 [==============================] - 2s 167ms/step


In [54]:
print("accuracy_score : ",accuracy_score(label, np.round(labels)))
print("recall_score : ", recall_score(label, np.round(labels),average='macro'))
print("precision_score : ",precision_score(label, np.round(labels),average='macro'))
print("f1_score : ",f1_score(label, np.round(labels),average='macro'))

accuracy_score :  0.9302325581395349
recall_score :  0.9302325581395349
precision_score :  0.9562167729930888
f1_score :  0.9406220236035329


In [49]:
def get_recall(y_true,y_scores,threshold):
    predict_positive_num = len(y_scores[y_scores >= threshold])
    tp = len( [x for x in y_true[:predict_positive_num] if x == 1] )
    ground_truth  = len(y_true[y_true==1])
    recall = tp / ground_truth
    return recall

def get_precision(y_true,y_scores,threshold):
    predict_positive_num = len(y_scores[y_scores >= threshold])
    tp = len( [x for x in y_true[:predict_positive_num] if x == 1] )
    fp = len( [x for x in y_true[:predict_positive_num] if x == 0] )
    precision = tp / (tp + fp) 
    return precision 

def recall_precision_plot(y_true, y_scores):
    recall, precision = [] , []

    for _ in y_scores: # y_scores 를 thresholds 처럼 사용했음
        recall.append(get_recall(y_true, y_scores, _ ))
        precision.append(get_precision(y_true,y_scores,_))

    fig = plt.figure(figsize=(9, 6))

    #3d container
    ax = plt.axes(projection = '3d')
    #3d scatter plot
    ax.plot3D(recall, y_scores, precision)
    ax.scatter3D(recall,y_scores,precision)
    #give labels
    ax.set_xlabel('Recall')
    ax.set_ylabel('Thresholds')
    ax.set_zlabel('Precision')
    ax.set_title('Precision-Recall Curve 3D')  
    plt.show()
    
    fig = plt.figure(figsize = (9,6))
    plt.plot(recall, precision)
    plt.scatter(recall,precision)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve 2D')
    plt.show()
    
recall_precision_plot(label,np.round(labels))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [23]:
label, np.round(labels)

(array([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]], dtype=float32),
 array([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 

In [7]:
base_model.trainable = True
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
checkpoint = ModelCheckpoint("checkpoints/vgg16_12.h5", monitor='accuracy', verbose=1, save_best_only=True )
early = EarlyStopping(monitor='accuracy', min_delta=0, patience=10, verbose=1, mode='auto')

history = model.fit(
        train_data,
        callbacks=[checkpoint, early],
        epochs=100,
        batch_size=32,
        verbose=1)

Epoch 1/100


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential_1/dense/MatMul/MatMul_1' defined at (most recent call last):
    File "C:\Users\pc\anaconda3\envs\gpu-py\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\pc\anaconda3\envs\gpu-py\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\pc\AppData\Roaming\Python\Python38\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\pc\anaconda3\envs\gpu-py\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\pc\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelapp.py", line 612, in start
      self.io_loop.start()
    File "C:\Users\pc\anaconda3\envs\gpu-py\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\p

In [ ]:
model.evaluate(train_data)

In [8]:
from keras.models import load_model
model = load_model('room.h5')

In [89]:
class_labels = ["Casual","Classic","Modern","Natural"]

In [93]:
#데이터 프레임 만들고 예측
df = pd.DataFrame(columns=["Photo","Casual","Classic","Modern","Natural"])

test_path = d

for i in test_path:
    test_data = train_datagen.flow_from_directory(
        i,
        target_size=image_size,
        shuffle=True,
        batch_size=32,
        class_mode = 'categorical' #2d 원핫인코딩
    )
    print(i)
    batch = next(test_data)
    images= batch[0]# 0번 이미지데이터 1번 레이블
    labels = model.predict(images)    
    sum_Caual=0
    sum_Classic=0
    sum_Modern=0
    sum_Natural=0

    for i in range(11) :
        if i !=10:
            # if np.argmax(labels[i])>0.5:

            sum_Caual +=round(labels[i][0]*100)
            sum_Classic +=round(labels[i][1]*100)
            sum_Modern +=round(labels[i][2]*100)
            sum_Natural +=round(labels[i][3]*100)
            dict_a={
                "Photo" : i,
                "Casual" :  f"{round(labels[i][0]*100,2)}%",
                "Classic" : f"{round(labels[i][1]*100,2)}%",
                "Modern" :  f"{round(labels[i][2]*100,2)}%",
                "Natural" :  f"{round(labels[i][3]*100,2)}%"
            }

        else:
              dict_a={
                "Photo" : "sum",
                "Casual" :  f"{sum_Caual/10}%",
                "Classic" : f"{sum_Classic/10}%",
                "Modern" :  f"{sum_Modern/10}%",
                "Natural" :  f"{sum_Natural/10}%"        
            }


        df = df.append(dict_a,ignore_index = True)

Found 10 images belonging to 1 classes.
C:\Users\pc\Desktop\0-200\0-200\plus_url0
1/1 [==============================] - 0s 53ms/step
Found 10 images belonging to 1 classes.
C:\Users\pc\Desktop\0-200\0-200\plus_url1
1/1 [==============================] - 0s 35ms/step
Found 10 images belonging to 1 classes.
C:\Users\pc\Desktop\0-200\0-200\plus_url10
1/1 [==============================] - 0s 31ms/step
Found 10 images belonging to 1 classes.
C:\Users\pc\Desktop\0-200\0-200\plus_url100
1/1 [==============================] - 0s 29ms/step
Found 10 images belonging to 1 classes.
C:\Users\pc\Desktop\0-200\0-200\plus_url101
1/1 [==============================] - 0s 31ms/step
Found 10 images belonging to 1 classes.
C:\Users\pc\Desktop\0-200\0-200\plus_url102
1/1 [==============================] - 0s 29ms/step
Found 10 images belonging to 1 classes.
C:\Users\pc\Desktop\0-200\0-200\plus_url103
1/1 [==============================] - 0s 30ms/step
Found 10 images belonging to 1 classes.
C:\Users\pc\D

In [94]:
df

,Photo,Casual,Classic,Modern,Natural
0,0,0.09%,0.01%,94.91%,4.98%
1,1,31.91%,0.07%,68.0%,0.02%
2,2,0.22%,0.0%,99.57%,0.2%
3,3,19.14%,53.23%,27.34%,0.29%
4,4,0.7%,0.63%,96.56%,2.11%
...,...,...,...,...,...
2195,6,3.02%,2.1%,1.07%,93.81%
2196,7,0.44%,86.33%,0.05%,13.17%
2197,8,98.74%,0.7%,0.43%,0.14%
2198,9,0.33%,9.12%,85.71%,4.83%


In [75]:
df.to_csv("plus-800-1000예측.csv")

In [76]:
df2=df[df["Photo"]=="sum"].reset_index()

In [77]:
df2_avg = df2.drop(columns=["index","Photo"])

In [78]:
del_num=[ 819,
 837,
 838,
 874,
 898,
 901,
 907,
 928,
 936,
 953,
 954,
 955,
 957,
 959,
 965,
 972,
 974,
 975,
 979,
 983,
 994,
 996,]
del_num2 = [i-800 for i in del_num]

In [79]:
# 테스트 데이터 프레임

# # 각 row별 가장 큰 값에 해당하는 컬럼명으로 새로운 컬럼생성
df2_avg['max_pre'] = [ df2_avg.columns[ np.argmax( [float(data[:-1]) for data in row] ) ] for row in df2_avg.values ]
df2_avg.drop(index=del_num2, inplace = True, axis=0)
df2_avg.reset_index(inplace = True)
df2_avg.drop(columns=["index"],inplace =True)
df2_avg

,Casual,Classic,Modern,Natural,max_pre
0,11.7%,0.2%,83.3%,4.9%,Modern
1,28.3%,10.7%,56.0%,4.9%,Modern
2,16.7%,4.0%,74.1%,5.2%,Modern
3,20.4%,34.3%,20.0%,25.1%,Classic
4,19.7%,20.0%,58.1%,2.4%,Modern
...,...,...,...,...,...
173,42.3%,3.6%,52.9%,1.2%,Modern
174,39.4%,1.3%,53.6%,5.7%,Modern
175,33.1%,25.4%,40.3%,1.3%,Modern
176,32.6%,13.1%,49.3%,4.8%,Modern


In [80]:
df2_avg.to_csv("plus-800-1000예측.csv-평균.csv")

In [81]:
print("Modern : ",len(df2_avg[df2_avg["max_pre"]=="Modern"]))
print("Natural : ",len(df2_avg[df2_avg["max_pre"]=="Natural"]))
print("Casual : ",len(df2_avg[df2_avg["max_pre"]=="Casual"]))
print("Classic : ",len(df2_avg[df2_avg["max_pre"]=="Classic"]))

Modern :  105
Natural :  10
Casual :  47
Classic :  16


In [82]:
#리뷰
a =pd.read_csv("plus_comment.csv")
# a=a[:-1]
a.drop(columns=["Unnamed: 0.1","Unnamed: 0"], inplace = True)
a

,숙소_형태,위치,가격,최대_인원수,침실개수,침대개수,욕실개수,소개글,댓글,댓글개수,평점
0,공동주택전체,"세비아, 안달루시아(Andalucía), 스페인",221736,4,1.0,1.0,1.0,['리모델링한 아파트의 옥상 파티오에서 일광욕을 즐기거나 공용 옥상 수영장에서 수영...,"['주변은 공사 한다 시끄러웠고, 뜨거운 물이 갑자기 안나와서 고생했다.', '숙소...",228,4.89
1,공동주택전체,"Lisboa, 포르투갈",172842,4,1.0,4.0,1.0,['리스본 중심부에 위치한 이 독특한 아파트의 전용 발코니에서 테호강의 일몰을 감상...,['단연코 지금껏 이용해 본 에어비앤비 숙소 중 가장 훌륭했습니다. 높은 곳에 위치...,276,4.92
2,공동주택전체,"멕시코시티, Ciudad de México, 멕시코",981756,6,3.0,4.0,3.5,['인증된 회사가 게스트 간의 숙소 소독 및 살균.신청일과 함께 서비스 인증서.선형...,['멋진 숙소입니다. 훌륭한 지역에 있으며 매우 깨끗하고 넓습니다. 연중무휴 경비원...,115,4.85
3,저택의개인실,"세비아, 안달루시아(Andalucía), 스페인",187404,2,1.0,2.0,1.0,['안달루시아 수도 한가운데 있는 궁전에서 깨어나고 싶으신가요? 기랄다가 내려다보이...,['너무 좋습니다블랑카는 매우 친절하고 룸 상태도 너무 좋고 불만을 갖을것이 하나 ...,341,4.92
4,공동주택전체,"부다페스트(Budapest), 헝가리",186620,2,1.0,1.0,1.0,['높이 솟은 커스텀 빌트인 침대에서 영화를 감상해보세요. 오페라에서 우아한 저녁을...,"['이틀 머문것이 아쉬운 좋은 숙소입니다..', '미할리와 쉽게 의사소통할 수 있었...",158,4.88
...,...,...,...,...,...,...,...,...,...,...,...
3489,저택전체,"Seminyak , 발리, 인도네시아",1005204,10,5.0,11.0,5.0,"['이 미적 명작에서 감각을 즐겨보세요. 엄선된 인테리어와 가구, 개별적으로 디자인...","['Airbnb의 특성상 호스트와의 커뮤니케이션이 아주 중요한데, 호스트가 직접 하...",112,4.95
3490,공동주택전체,"Praha 1, Hlavní město Praha, 체코",158476,6,2.0,4.0,1.0,"['채광창, 바닥의 오리엔탈 러그, 나무로 마감된 천장이 있는 주방에서 자연 채광을...","['숙소가 꼭대기 층에 위치해 있어서 걸어올라가는 데 힘들었지만, 창문 밖으로 보이...",271,4.83
3491,콘도전체,"바르셀로나, 카탈루냐, 스페인",111813,6,4.0,5.0,2.0,['고층 아파트의 햇살이 잘 드는 발코니에서 옥상을 바라보세요. 리모델링한 복고풍 ...,"['정말 좋았습니다ㅈ', '비행기 3시간 지연으로 관리인분께연락드렸고 바로답을 받았...",468,4.80
3492,로프트전체,"Lake Placid, 뉴욕, 미국",609292,4,1.0,2.0,1.0,"['창문, 스카이라이트, 프라이빗 데크에서 높은 봉우리와 올림픽 빌리지의 전망을 감...",['훌륭한 위치와 주차. 체크인 및 체크아웃이 간편합니다. 아름다운 로프트로 잘 꾸...,205,4.95


In [53]:
a=a.fillna(0)
a.isnull().sum()

숙소_형태     0
위치        0
가격        0
최대_인원수    0
침실개수      0
침대개수      0
욕실개수      0
소개글       0
댓글        0
댓글개수      0
평점        0
dtype: int64

In [83]:
df_800_1000 = a[800:1000]

In [84]:
df_800_1000.drop(index=del_num, inplace = True, axis=0)
df_800_1000.reset_index(inplace = True)
df_800_1000.drop(columns=["index"],inplace =True)

In [85]:
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.max_columns', 50)

In [86]:
df_800_1000 = pd.concat([df_800_1000,df2_avg],axis=1)
df_800_1000

,숙소_형태,위치,가격,최대_인원수,침실개수,침대개수,욕실개수,소개글,댓글,댓글개수,평점,Casual,Classic,Modern,Natural,max_pre
0,공동주택전체,"Athina, 그리스",143310,4,NaN,2.0,1.0,['고대와 유서 깊은 도시 중심가에서 대중문화와 현대 예술의 오아시스를 만나보세요....,"['숙소는 위치도 좋고 깨끗하고 예뻤습니다 !! 1층이라고 해서 걱정했는데, 한국으...",423,4.87,11.7%,0.2%,83.3%,4.9%,Modern
1,로프트전체,"발렌시아(Valencia), Comunidad Valenciana, 스페인",144883,2,1.0,1.0,1.0,['현대적인 인테리어의 VT-43639-V 2층 로프트. 커다란 창문을 통해 레반테...,['마르코스는 매우 도움이 되었고 아파트에서 바라보는 풍경이 훌륭했습니다. 체크인과...,371,4.92,28.3%,10.7%,56.0%,4.9%,Modern
2,공동주택전체,"밀라노, Lombardia, 이탈리아",271598,6,2.0,2.0,1.0,"['몽환적인 분위기와 인스타그램을 즐길 수 있는 코너가 마음에 드신다면, 아파트의 ...","['밀라노에서 중심가입니다 생각보다는 크지 않지만 깔끔합니다', '없음', '멋진 ...",102,4.79,16.7%,4.0%,74.1%,5.2%,Modern
3,공동주택전체,"Warszawa, mazowieckie, 폴란드",130396,4,2.0,2.0,1.0,['바르샤바의 역사적인 구시가지 한가운데에 위치한 이 매력적이고 아늑한 아파트는 폴...,"['간편한 체크인과 훌륭한 위치 - 추천합니다.', '아파트의 위치는 구시가지 한복...",189,4.87,20.4%,34.3%,20.0%,25.1%,Classic
4,공동주택전체,"Praha 8, Hlavní město Praha, 체코",93032,3,1.0,2.0,1.0,"['파스텔 색상의 커튼, 베개, 헤드보드에서부터 화분과 하트 모양의 분홍색에 이르기...",['두명이서 쓰기에 완벽했어요! 위치는 중심부에서 조금 떨어져 있는 정도인데 천천히...,166,4.73,19.7%,20.0%,58.1%,2.4%,Modern
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,공동주택전체,"파리, Île-de-France, 프랑스",173880,2,1.0,1.0,1.0,['영화의 세계에 헌신하는 이 주택으로 7번째 예술 작품을 감상해보세요. 이 이전 ...,['공간이 분리되어있어 좋았습니다. 티비가없어서 불편했고 스크린이있지만 사용하지못했...,307,4.85,42.3%,3.6%,52.9%,1.2%,Modern
174,공동주택전체,"Saint-Cloud, Île-de-France, 프랑스",160003,4,1.0,1.0,1.5,"['아파트는 꽃과 나무가 가득한 조용한 동네에 있으며, 넓고 밝은 68제곱미터입니다...",['이 후기는 어디서부터 시작해야 할지 모르겠지만 아파트는 하위 기준이며 에어비앤비...,128,4.79,39.4%,1.3%,53.6%,5.7%,Modern
175,공동주택전체,"올비아, Sardegna, 이탈리아",157483,4,1.0,2.0,1.0,['세련된 가구에서 창의성의 여유를 느껴보세요. 독특한 솔루션으로 편안함을 극대화하...,['집이 이뻐요! 마리아는 매우 세심하고 도움이 되었으며 커뮤니케이션이 훌륭했습니다...,111,4.97,33.1%,25.4%,40.3%,1.3%,Modern
176,공동주택전체,"말라가, 안달루시아(Andalucía), 스페인",108664,4,2.0,3.0,1.0,"['조심스러운 인테리어부터 작은 디테일까지, 야외 조명이 모든 숙박을 비추는 대형 ...",['깨끗하고 편안한 숙소였습니다. 도심의 주요 관광지까지 5분 정도에 위치에 걸어가...,122,4.78,32.6%,13.1%,49.3%,4.8%,Modern


In [87]:
df_2100_2400.to_csv("plus-800-1000_리뷰+사진예측.csv")